# **Obtención de información de simetrías para dataset de 134 Kmoléculas**

## Imports 

In [ ]:
import pymsym
import numpy as np
import os
from datetime import datetime
import pandas as pd
import openpyxl 

## Diccionario de moléculas predefinidas
`MOLECULES`

In [ ]:
# Diccionario de moléculas predefinidas con estructuras XYZ
#
# Este diccionario contiene una variedad de moléculas con diferentes características
# de simetría y complejidad estructural:
#
# 1. Moléculas altamente simétricas:
#    - SF6 (grupo puntual Oh): estructura octaédrica perfecta
#    - Benceno (D6h): anillo aromático plano
#    - Metano (Td): estructura tetraédrica
#    - CO2 (D∞h): molécula lineal
#
# 2. Moléculas con simetría intermedia:
#    - Amoníaco (C3v): estructura piramidal
#    - Ciclopropano (D3h): anillo triangular
#    - Eteno (D2h): doble enlace con geometría plana
#    - Ciclobutadieno (D4h): anillo cuadrado
#
# 3. Moléculas con baja simetría o sin simetría:
#    - Etanol (C1): sin elementos de simetría
#    - Ibuprofen (C1): molécula orgánica compleja sin simetría
#    - Propeno (Cs): solo un plano de simetría
#
# 4. Moléculas clasificadas por tamaño:
#    Pequeñas (2-4 átomos):
#    - Agua (H2O)
#    - Amoníaco (NH3)
#    - Dióxido de carbono (CO2)
#
#    Medianas (5-12 átomos):
#    - Benceno (C6H6)
#    - Ciclopropano (C3H6)
#    - Eteno (C2H4)
#
#    Grandes (>12 átomos):
#    - Ibuprofen (C13H18O2)
#
# Cada estructura está en formato XYZ estándar:
# - Primera línea: número de átomos
# - Segunda línea: comentario/nombre
# - Siguientes líneas: símbolo atómico y coordenadas x, y, z en Angstroms

MOLECULES = {
    'water': """3
H2O molecule
O     0.000000     0.000000     0.000000
H     0.758602     0.000000     0.504284
H    -0.758602     0.000000     0.504284""",

    'methane': """5
CH4 molecule
C     0.000000     0.000000     0.000000
H     0.629118     0.629118     0.629118
H    -0.629118    -0.629118     0.629118
H     0.629118    -0.629118    -0.629118
H    -0.629118     0.629118    -0.629118""",

    'ammonia': """4
NH3 molecule
N     0.000000     0.000000     0.000000
H     0.947260     0.000000     0.337440
H    -0.473630     0.820458     0.337440
H    -0.473630    -0.820458     0.337440""",

    'ethanol': """9
C2H5OH molecule - sin simetría
C    -0.012540    -0.566690     0.000000
C     1.366770     0.022570     0.000000
O    -0.983400     0.477460     0.000000
H    -0.162870    -1.196370     0.890600
H    -0.162870    -1.196370    -0.890600
H     1.986680    -0.882900     0.000000
H     1.558060     0.636780     0.890600
H     1.558060     0.636780    -0.890600
H    -1.869220     0.094100     0.000000""",

    'benzene': """12
C6H6 molecule
C     0.000000     1.396792     0.000000
C     1.209502     0.698396     0.000000
C     1.209502    -0.698396     0.000000
C     0.000000    -1.396792     0.000000
C    -1.209502    -0.698396     0.000000
C    -1.209502     0.698396     0.000000
H     0.000000     2.484216     0.000000
H     2.151390     1.242108     0.000000
H     2.151390    -1.242108     0.000000
H     0.000000    -2.484216     0.000000
H    -2.151390    -1.242108     0.000000
H    -2.151390     1.242108     0.000000""",

    'carbon_dioxide': """3
CO2 molecule
C     0.000000     0.000000     0.000000
O     0.000000     0.000000     1.162436
O     0.000000     0.000000    -1.162436""",

    'trans_butene': """12
trans-butene molecule
C    -1.944270     0.000000     0.000000
C    -0.749640     0.000000     0.000000
C     0.749640     0.000000     0.000000
C     1.944270     0.000000     0.000000
H    -2.481635     0.935825     0.000000
H    -2.481635    -0.935825     0.000000
H     0.935825     1.028615     0.000000
H    -0.935825    -1.028615     0.000000
H     2.481635     0.935825     0.000000
H     2.481635    -0.935825     0.000000
H    -0.935825     1.028615     0.000000
H     0.935825    -1.028615     0.000000""",

    'cyclopropane': """9
Cyclopropane molecule
C     0.000000     0.872927     0.000000
C     0.756252    -0.436463     0.000000
C    -0.756252    -0.436463     0.000000
H     0.000000     1.454927     0.907837
H     0.000000     1.454927    -0.907837
H     1.259952    -0.727463     0.907837
H     1.259952    -0.727463    -0.907837
H    -1.259952    -0.727463     0.907837
H    -1.259952    -0.727463    -0.907837""",

    'ibuprofen': """33
Ibuprofen molecule - sin simetría
C    -1.203000    -0.833000     0.058000
C    -0.047000     0.163000     0.063000
C     1.278000    -0.610000     0.065000
C     2.513000     0.297000     0.070000
O     2.461000     1.518000     0.073000
O     3.705000    -0.359000     0.071000
C    -2.528000    -0.060000     0.056000
C    -2.852000     0.730000     1.167000
C    -4.075000     1.409000     1.166000
C    -4.999000     1.310000     0.123000
C    -4.696000     0.530000    -0.997000
C    -3.473000    -0.149000    -0.996000
C    -5.699000     0.421000    -2.126000
H    -1.271000    -1.452000     0.960000
H    -1.273000    -1.509000    -0.805000
H    -0.113000     0.812000     0.943000
H    -0.113000     0.812000    -0.817000
H     1.344000    -1.259000     0.945000
H     1.344000    -1.259000    -0.815000
H     4.459000     0.245000     0.074000
H    -2.137000     0.809000     1.981000
H    -4.302000     2.014000     2.039000
H    -5.957000     1.837000     0.122000
H    -3.246000    -0.754000    -1.869000
H    -5.204000     0.452000    -3.104000
H    -6.297000    -0.496000    -2.024000
H    -6.297000     1.338000    -2.024000""",

    'sulfur_hexafluoride': """7
SF6 molecule
S     0.000000     0.000000     0.000000
F     0.000000     0.000000     1.564000
F     0.000000     0.000000    -1.564000
F     0.000000     1.564000     0.000000
F     0.000000    -1.564000     0.000000
F     1.564000     0.000000     0.000000
F    -1.564000     0.000000     0.000000""",

    'ethene': """6
C2H4 molecule
C     0.000000     0.000000     0.665000
C     0.000000     0.000000    -0.665000
H     0.000000     0.922000     1.237000
H     0.000000    -0.922000     1.237000
H     0.000000     0.922000    -1.237000
H     0.000000    -0.922000    -1.237000""",

    'propene': """9
C3H6 molecule
C     0.000000     0.000000     0.000000
C     1.337000     0.000000     0.000000
C    -0.858000     1.213000     0.000000
H     2.093000    -0.767000     0.000000
H     1.753000     0.999000     0.000000
H    -0.441000    -0.985000     0.000000
H    -1.921000     1.003000     0.000000
H    -0.600000     1.811000     0.882000
H    -0.600000     1.811000    -0.882000""",

    'cyclobutadiene': """8
C4H4 molecule
C     0.707107     0.707107     0.000000
C    -0.707107     0.707107     0.000000
C    -0.707107    -0.707107     0.000000
C     0.707107    -0.707107     0.000000
H     1.420000     1.420000     0.000000
H    -1.420000     1.420000     0.000000
H    -1.420000    -1.420000     0.000000
H     1.420000    -1.420000     0.000000"""
}

## Diccionario para convertir símbolos atómicos a números atómicos
`ATOMIC_NUMBERS_DICT`

In [ ]:
# Diccionario para convertir símbolos atómicos a números atómicos
ATOMIC_NUMBERS_DICT = {
    'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8, 'F': 9, 'Ne': 10,
    'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15, 'S': 16, 'Cl': 17, 'Ar': 18,
    'K': 19, 'Ca': 20, 'Br': 35, 'I': 53
}

## Lee un archivo XYZ y devuelve su contenido como string
`read_xyz_file(filepath)`

In [ ]:
def read_xyz_file(filepath):
    """
    Lee un archivo XYZ y devuelve su contenido como string.

    Args:
        filepath (str): Ruta al archivo XYZ

    Returns:
        str: Contenido del archivo XYZ
    """
    try:
        with open(filepath, 'r') as file:
            return file.read()
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo: {filepath}")
    except Exception as e:
        raise Exception(f"Error al leer el archivo: {str(e)}")


## Analiza la simetría de una molécula a partir de su estructura XYZ en formato string
`analyze_molecule_symmetry(xyz_string, molecule_name="Unknown")`

In [ ]:
def analyze_molecule_symmetry(xyz_string, molecule_name="Unknown"):
    """
    Analiza la simetría de una molécula a partir de su estructura XYZ en formato string.

    Args:
        xyz_string (str): Estructura molecular en formato XYZ como string
        molecule_name (str): Nombre identificativo de la molécula

    Returns:
        dict: Diccionario con toda la información de simetría
    """
    # Parsear el string XYZ
    lines = xyz_string.strip().split('\n')
    n_atoms = int(lines[0])
    comment = lines[1].strip()

    # Extraer símbolos atómicos y coordenadas
    atomic_numbers = []
    coordinates = []
    symbols = []

    for line in lines[2:2+n_atoms]:
        parts = line.split()
        symbol = parts[0]
        symbols.append(symbol)
        if symbol not in ATOMIC_NUMBERS_DICT:
            raise ValueError(f"Átomo no reconocido: {symbol}")
        atomic_numbers.append(ATOMIC_NUMBERS_DICT[symbol])
        coords = [float(x) for x in parts[1:4]]
        coordinates.append(coords)

    coordinates = np.array(coordinates)

    # Obtener toda la información de simetría disponible
    symmetry_info = {
        'molecule_name': molecule_name,
        'analysis_timestamp': datetime.now().isoformat(),
        'structure_info': {
            'n_atoms': n_atoms,
            'comment': comment,
            'coordinates': coordinates.tolist(),
            'atomic_numbers': atomic_numbers,
            'symbols': symbols
        },
        'composition': {symbol: symbols.count(symbol) for symbol in set(symbols)},
        'symmetry_analysis': {
            'point_group': pymsym.get_point_group(atomic_numbers, coordinates),
            'symmetry_number': pymsym.get_symmetry_number(atomic_numbers, coordinates)
        }
    }

    # Intentar obtener información adicional si está disponible en pymsym
    try:
        pg = pymsym.PointGroup(coordinates)  # Algunos métodos adicionales si están disponibles
        additional_info = {
            'order': pg.order if hasattr(pg, 'order') else None,
            'generators': pg.generators if hasattr(pg, 'generators') else None,
            'symmetry_operations': pg.symmetry_operations if hasattr(pg, 'symmetry_operations') else None
        }
        symmetry_info['symmetry_analysis'].update(additional_info)
    except:
        pass  # Si no está disponible, continuamos sin esta información adicional

    return symmetry_info

## Analiza la simetría de una molécula desde un archivo XYZ

In [ ]:
def analyze_from_file(filepath):
    """
    Analiza la simetría de una molécula desde un archivo XYZ.

    Args:
        filepath (str): Ruta al archivo XYZ

    Returns:
        dict: Información completa de simetría
    """
    try:
        xyz_string = read_xyz_file(filepath)
        molecule_name = os.path.basename(filepath).replace('.xyz', '')
        results = analyze_molecule_symmetry(xyz_string, molecule_name)

        # Imprimir resumen
        print(f"\nAnálisis de simetría para {molecule_name}:")
        print(f"Número de átomos: {results['structure_info']['n_atoms']}")
        print("Composición molecular:")
        for atom, count in results['composition'].items():
            print(f"  {atom}: {count}")
        print(f"Grupo puntual: {results['symmetry_analysis']['point_group']}")
        print(f"Número de simetría: {results['symmetry_analysis']['symmetry_number']}")

        return results

    except Exception as e:
        print(f"Error al analizar el archivo: {str(e)}")
        return None


## Analiza todos los archivos XYZ en un directorio
`analyze_multiple_files(directory, extension='.xyz')`

In [ ]:
def analyze_multiple_files(directory, extension='.xyz'):
    """
    Analiza todos los archivos XYZ en un directorio.

    Args:
        directory (str): Ruta al directorio
        extension (str): Extensión de los archivos a analizar (por defecto '.xyz')

    Returns:
        dict: Diccionario con los resultados de todas las moléculas
    """
    try:
        files = [f for f in os.listdir(directory) if f.endswith(extension)]
        if not files:
            print(f"No se encontraron archivos {extension} en el directorio.")
            return {}

        results = {}
        for file in files:
            filepath = os.path.join(directory, file)
            results[file] = analyze_from_file(filepath)

        return results

    except Exception as e:
        print(f"Error al analizar el directorio: {str(e)}")
        return None

## Analiza una de las moléculas predefinidas
`analyze_predefined_molecule(molecule_name)`

In [ ]:
def analyze_predefined_molecule(molecule_name):
    """
    Analiza una de las moléculas predefinidas.

    Args:
        molecule_name (str): Nombre de la molécula ('water', 'methane', o 'caffeine')

    Returns:
        dict: Información completa de simetría
    """
    if molecule_name.lower() not in MOLECULES:
        raise ValueError(f"Molécula no encontrada. Disponibles: {', '.join(MOLECULES.keys())}")

    xyz_string = MOLECULES[molecule_name.lower()]
    return analyze_molecule_symmetry(xyz_string, molecule_name)

## Ejemplo de uso

In [ ]:
# 1. Analizar todas las moléculas predefinidas
results_dict = {}
print("Analizando moléculas predefinidas...")
for name in MOLECULES.keys():
    results_dict[name] = analyze_predefined_molecule(name)

    # Imprimir resumen para cada molécula
    print(f"\nMolécula: {name}")
    print(f"Grupo puntual: {results_dict[name]['symmetry_analysis']['point_group']}")
    print(f"Número de simetría: {results_dict[name]['symmetry_analysis']['symmetry_number']}")
    print("Composición:", results_dict[name]['composition'])

# 2. Para analizar un archivo XYZ:
# result = analyze_from_file('ruta/a/tu/molecula.xyz')
result = analyze_from_file('/home/javilukt/aaaproyectos/phd/grupos-simetria/data/raw/dsgdb9nsd.xyz/dsgdb9nsd_000001.xyz')
print(result)

# 3. Para analizar todos los archivos XYZ en un directorio:
results = analyze_multiple_files('/home/javilukt/aaaproyectos/phd/grupos-simetria/data/raw/dsgdb9nsd.xyz')

## Lectura de todos los xyz de un directorio (134 Kmol) y exportación a un archivo para rescate

In [ ]:
# prompt: Y si en vez de enviarlo a un json lo envio a un excel?

def save_results(results, output_directory):
    """Saves the results dictionary to an Excel file in the specified directory.

    Args:
        results (dict): The dictionary containing the symmetry analysis results.
        output_directory (str): The directory to save the Excel file to.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = os.path.join(output_directory, f"symmetry_analysis_results_{timestamp}.xlsx")

    try:
        # Create a list to store dataframes for each molecule
        all_dfs = []

        for filename, result in results.items():
            if result:  # Check if result is not None
                # Convert the result dictionary to a dataframe
                df = pd.DataFrame([result])
                # Add filename as a new column
                df['filename'] = filename
                all_dfs.append(df)

        if all_dfs:
            # Concatenate all dataframes into a single dataframe
            final_df = pd.concat(all_dfs, ignore_index=True)
            # Save to Excel file
            final_df.to_excel(output_filename, index=False)  # index=False removes row indices
            print(f"Results saved to {output_filename}")
        else:
            print("No valid results to save to Excel.")

    except Exception as e:
        print(f"Error saving results: {e}")


# Example usage
output_path = '/home/javilukt/aaaproyectos/phd/grupos-simetria/data/processed'

if results:
    save_results(results, output_path)
else:
    print("The 'results' dictionary is empty. Nothing to save.")

In [ ]:
# prompt: Quiero preparar una estructura con los datos que se obtienen de todos los xyz de un directorio llamado /midirectorio y guardarlo en /midirectorioout para poder luego recuperarlo

import json
import os

def save_results(results, output_directory):
    """Saves the results dictionary to a JSON file in the specified directory.

    Args:
        results (dict): The dictionary containing the symmetry analysis results.
        output_directory (str): The directory to save the JSON file to.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)  # Create the output directory if it doesn't exist

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # Get current timestamp
    output_filename = os.path.join(output_directory, f"symmetry_analysis_results_{timestamp}.json")
    
    try:
        with open(output_filename, 'w') as f:
            json.dump(results, f, indent=4)  # Save to JSON file with indentation
        print(f"Results saved to {output_filename}")
    except Exception as e:
        print(f"Error saving results: {e}")


# Example usage (assuming 'results' is your results dictionary):
output_path = '/home/javilukt/aaaproyectos/phd/grupos-simetria/data/processed' # Change to your desired output directory

if results: # Check if the 'results' dictionary has been populated
    save_results(results, output_path)
else:
    print("The 'results' dictionary is empty. Nothing to save.")


In [ ]:
# prompt: Dame ahora lo necesario para rescartarlo y poderlo luego usar con pandas

import pandas as pd
import json
import os

def load_results_to_dataframe(filepath):
    """Loads symmetry analysis results from a JSON file into a Pandas DataFrame.

    Args:
        filepath (str): Path to the JSON file.

    Returns:
        pandas.DataFrame: DataFrame containing the symmetry analysis results, or None if an error occurs.
    """
    try:
        with open(filepath, 'r') as f:
            results = json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    
    # Convert the results to a list of dictionaries
    data = []
    for filename, analysis in results.items():
        if analysis:  # Check if the analysis for this file is not None
            row = {
                'filename': filename,
                'molecule_name': analysis.get('molecule_name', 'N/A'),
                'point_group': analysis['symmetry_analysis'].get('point_group', 'N/A'),
                'symmetry_number': analysis['symmetry_analysis'].get('symmetry_number', 'N/A'),
                'n_atoms': analysis['structure_info'].get('n_atoms', 'N/A'),
                # Add other relevant fields as needed
            }
            data.append(row)
        else:
            print(f"Skipping empty analysis for file {filename}")

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


# Example Usage:
filepath = '/content/drive/My Drive/PhD/datos/xyzsout/symmetry_analysis_results_*.json' # Update with the actual filepath
# Find the latest JSON file in the directory
import glob
list_of_files = glob.glob(filepath) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
# Load the data
df = load_results_to_dataframe(latest_file)

if df is not None:
df
    # Now you can work with the DataFrame (e.g., df.head(), df.describe(), etc.)